# 3. 협업 필터링 추천 시스템
* 실제 취향을 고려한 개인화 추천 시스템
* 이러한 아이디어로 만들어진 추천 알고리즘이 협업 필터링,   
  콜라보레이티브 필터링 CF
* 어떤 아이템에 대해 비슷한 취향을 가진 사람들은,  
  다른 아이템 또한 비슷한 취향을 가질 것이다

## 3.1. 협업 필터링의 원리
* 추천 대상과 취향이 비슷한 사람들의 집단이 먼저 만들어져 있다
* 취향이 비슷한 사람들을 찾아내기만 하면,  
  그 사람들이 좋아하는 아이템을 추천하면 된다  
* 유사도 계산 필요

## 3.2. 유사도 지표
* CF에서 사용자 간 유사도를 구하는 것이 핵심

### 3.2.1. 상관계수
* 가장 이해하기 쉬운 유사도
* -1과 +1 사이의 값  
  * -1이면 부적 상관 관계, 음의 상관 관계 (우하향?)
  * +1이면 양의 상관 관계 (우상향?)
  * 0이면 지표가 점으로 나타나 연속적이지 않음, 유사하지 않게 나타남, 유사도가 0이라는 뜻  
  * 그런데 대문자 U 모양으로 쿼드러틱하게 나타나는 경우도 유사도가 0으로 표현된다
  * 단순히 상관 관계가 0이라고 해서 유사도가 없다고 생각하면 안 되고,  
    데이터를 실제로 비주얼라이제이션 해보거나 해서 정확한 인사이트를 얻어야 한다
* 평가 자료가 연속형인 경우 가장 이해하기 쉬운 유사도

### 3.2.2. 코사인 유사도
* 협업 필터링에서 가장 널리 쓰이는 유사도
* 각 아이템 → 하나의 차원, 사용자의 평가값 → 좌표값
  * 예를 들면 x축은 아이템A, y축은 아이템B를 나타내고  
    점 C(x1, y1)에서 x1과 y1은 사용자C가 아이템A와 아이템B 각각에 대해 평가한 평가값을 나타낸다
* 두 사용자의 평가값 유사하다  
  → 코사인theta에서 theta는 작아지고 코사인 값은 커짐  
  (theta : 점A와 점B를 각각 원점과 이은 직선 사이의 각)
* -1과 +1 사이의 값
* 데이터가 이진값이라면(샀다 안 샀다/봤다 안 봤다) 타니모토 계수 사용 권장  

### 3.2.3. 자카드 계수
* 타니모토 계수의 변형
* 이진수 데이터라면 협업 필터링에서 좋은 결과 나타난다

## 3.3. 기본 CF 알고리즘
> 1. 모든 사용자 간 평가의 유사도 계산  
> 2. 추천 대상과 다른 사용자 간 유사도 추출
> 3. 추천 대상이 평가하지 않은 아이템에 대한 예상 평가값 계산   
>  → 평가값 = 다른 사용자 평가 × 다른 사용자 유사도
> 4. 아이템 중에서 예상 평가값 가장 높은 N개 추천  



In [ ]:
import os
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split

In [ ]:
# 데이터 불러오기
base_src = "drive/MyDrive/RecoSys/python-recosys/Data"

## user 데이터
u_user_src = os.path.join(base_src, "u.user")
u_cols = ["user_id", "age", "sex", "occupation", "zipcode"]
users = pd.read_csv(u_user_src, sep = "|", names = u_cols, encoding = "latin-1")
users = users.set_index("user_id")

## movie 데이터
u_item_src = os.path.join(base_src, "u.item")
i_cols = ['movie_id','title','release date','video release date',
          'IMDB URL', 'unknown', 'Action', 'Adventure','Animation',
          'ChildrenI\'s','Comedy','Crime','Documentary','Drama','Fantasy',
          'Film-Noir', 'Horror', 'Musical' ,'Mystery','Romance','Sci-Fi','Thriller', 'War','Western']
items = pd.read_csv(u_item_src, sep = "|", names = i_cols, encoding = "latin-1")
items = items.set_index("movie_id")

## ratings 데이터
u_ratings_src = os.path.join(base_src, "u.data")
r_cols = ["user_id", "movie_id", "rating", "timestamp"]
ratings = pd.read_csv(u_ratings_src, sep = "\t", names = r_cols, encoding = "latin-1")

In [ ]:
# 필요한 함수 정의

## 정확도 지표 RMSE - 낮을수록 정확
def RMSE(y_true, y_pred) :
  return np.sqrt(np.mean((np.array(y_true) - np.array(y_pred)) ** 2))

## score
def score(model) :
  # 테스트 데이터의 user_id와 movie_id의 쌍을 맞춰 튜플형 원소 리스트 데이터 생성
  id_pairs = zip(x_test["user_id"], x_test["movie_id"])

  # 모든 사용자-영화 쌍에 대해서 주어진 예측 모델에 의해 예측값 계산 및 리스트형 데이터 생성
  y_pred = np.array([model(user, movie) for (user, movie) in id_pairs])

  # 실제 평점값
  y_true = np.array(x_test["rating"])

  return RMSE(y_true, y_pred)

In [ ]:
# 데이터 셋 생성

x = ratings.copy()
y = ratings["user_id"]

x_train, x_test, y_train, y_test = train_test_split(x, y, test_size = 0.25, stratify = y)

ratings_matrix = x_train.pivot(index = "user_id", columns = "movie_id", values = "rating")

In [ ]:
print(x_train)
print(y_train)
print(ratings_matrix)

       user_id  movie_id  rating  timestamp
20083       90       151       2  891385190
78556      347       148       3  881652888
19132       43       879       4  876159838
17096      329       250       3  891656639
48036      429       928       2  882386849
...        ...       ...     ...        ...
20187      346       132       4  875261235
72946      684       217       2  875811965
37908      561       202       3  885808867
82895      125      1037       2  892839143
16369       13       377       1  882399219

[75000 rows x 4 columns]
20083     90
78556    347
19132     43
17096    329
48036    429
        ... 
20187    346
72946    684
37908    561
82895    125
16369     13
Name: user_id, Length: 75000, dtype: int64
movie_id  1     2     3     4     5     6     7     8     9     10    ...  \
user_id                                                               ...   
1          NaN   3.0   4.0   3.0   3.0   5.0   4.0   1.0   NaN   3.0  ...   
2          NaN   NaN   NaN   

In [ ]:
# 코사인 유사도 계산
from sklearn.metrics.pairwise import cosine_similarity

## 일단 코사인 유사도를 계산하기 위해
## ratings_matrix 복사하고 NaN값 0으로 대체
matrix_dummy = ratings_matrix.copy().fillna(0)

user_similarity = cosine_similarity(matrix_dummy, matrix_dummy)
user_similarity = pd.DataFrame(user_similarity, index = ratings_matrix.index, columns = ratings_matrix.index)

user_similarity

user_id,1,2,3,4,5,6,7,8,9,10,...,934,935,936,937,938,939,940,941,942,943
user_id,,,,,,,,,,,,,,,,,,,,,
1,1.000000,0.121875,0.019116,0.049772,0.286653,0.312590,0.321495,0.232803,0.058062,0.279873,...,0.243606,0.070810,0.205655,0.142205,0.150244,0.070972,0.242357,0.079187,0.132284,0.271730
2,0.121875,1.000000,0.116889,0.142336,0.037715,0.202656,0.105917,0.061447,0.067661,0.121975,...,0.090549,0.145555,0.223168,0.295365,0.232919,0.109563,0.148461,0.103090,0.138663,0.083169
3,0.019116,0.116889,1.000000,0.228683,0.026798,0.033430,0.046410,0.043342,0.000000,0.032269,...,0.032046,0.055326,0.098206,0.044598,0.087019,0.018704,0.128555,0.080731,0.131058,0.000000
4,0.049772,0.142336,0.228683,1.000000,0.000000,0.043111,0.072188,0.164630,0.000000,0.063462,...,0.043688,0.047565,0.084430,0.148856,0.151287,0.038593,0.150203,0.178474,0.076152,0.060155
5,0.286653,0.037715,0.026798,0.000000,1.000000,0.162825,0.258649,0.221122,0.038155,0.152344,...,0.295881,0.069897,0.081709,0.055820,0.096007,0.032834,0.170548,0.093558,0.123404,0.211947
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
939,0.070972,0.109563,0.018704,0.038593,0.032834,0.090762,0.110808,0.073144,0.000000,0.067393,...,0.050991,0.402265,0.205681,0.118829,0.262055,1.000000,0.046004,0.068511,0.039151,0.105537
940,0.242357,0.148461,0.128555,0.150203,0.170548,0.240248,0.196687,0.140521,0.028227,0.219506,...,0.234347,0.087884,0.130837,0.087108,0.126832,0.046004,1.000000,0.147151,0.184330,0.157605
941,0.079187,0.103090,0.080731,0.178474,0.093558,0.136322,0.054758,0.027061,0.134649,0.088880,...,0.039302,0.187347,0.283911,0.168161,0.221347,0.068511,0.147151,1.000000,0.068973,0.072304


In [ ]:
# 주어진 movie_id 영화의 가중평균 rating 계산하는 함수
def CF_simple(user_id, movie_id) :
  if movie_id in ratings_matrix.columns :
    # 해당 사용자와 다른 사용자 간의 유사성만 추출
    similarity_scores = user_similarity[user_id].copy()

    # 주어진 영화에 대한 다른 사용자들의 평가
    movie_ratings = ratings_matrix[movie_id].copy()

    # 주어진 영화에 대해 평가를 하지 않은 사용자들 가중평균에서 제외하기 위해
    # 해당 사용자들의 index 추출하고
    # 평가하지 않은 애들 제외시키기
    non_rating_idx = movie_ratings[movie_ratings.isnull()].index
    similarity_scores = similarity_scores.drop(non_rating_idx)
    movie_ratings = movie_ratings.dropna()

    # 주어진 영화에 대해 평가한 사용자들의 평점을 유사도로 가중평균한 유사도 similarity_scores
    # 주어진 영화에 대한 평점 movie_ratings를 가중평균하고
    # 전체 유사도에서 sum을 나누어주면 mean_rating을 얻을 수 있다
    mean_rating = np.dot(similarity_scores, movie_ratings) / similarity_scores.sum()

  # movie_id가 없으면
  else :
    mean_rating = 3.0

  return mean_rating

In [ ]:
# 정확도 계산
score(CF_simple)
## 약 1.010051245392655

1.010051245392655

## 3.4. 이웃을 고려한 CF
* 모든 사용자들과 유사도를 비교하는 게 아니고
* 정말로 유사도가 높은 사용자들을 선정해 예측치 판단


* 유사 집단을 정하는 방법 두 가지 KNN과 Thresholding

### 3.4.1. KNN : K Nearest Neighbors
* 유사도가 높은 K개 선정


### 3.4.2. Thresholding
* 미리 기준을 정해두고  해당 기준을 넘는다면 개수에 상관 없이 모두 채택
* 상관 계수 0.8 이상 또는 코사인 유사도 0.7 이상 등
* 일반적으로 KNN보다 정확

In [ ]:
import os
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.metrics.pairwise import cosine_similarity

In [ ]:
# 데이터 불러오기
base_src = "drive/MyDrive/RecoSys/python-recosys/Data"

## user 데이터
u_user_src = os.path.join(base_src, "u.user")
u_cols = ["user_id", "age", "sex", "occupation", "zipcode"]
users = pd.read_csv(u_user_src, sep = "|", names = u_cols, encoding = "latin-1")
users = users.set_index("user_id")

## movie 데이터
u_item_src = os.path.join(base_src, "u.item")
i_cols = ['movie_id','title','release date','video release date',
          'IMDB URL', 'unknown', 'Action', 'Adventure','Animation',
          'ChildrenI\'s','Comedy','Crime','Documentary','Drama','Fantasy',
          'Film-Noir', 'Horror', 'Musical' ,'Mystery','Romance','Sci-Fi','Thriller', 'War','Western']
movies = pd.read_csv(u_item_src, sep = "|", names = i_cols, encoding = "latin-1")
movies = movies.set_index("movie_id")

## ratings 데이터
u_ratings_src = os.path.join(base_src, "u.data")
r_cols = ["user_id", "movie_id", "rating", "timestamp"]
ratings = pd.read_csv(u_ratings_src, sep = "\t", names = r_cols, encoding = "latin-1")

In [ ]:
# 필요한 함수 정의

## 정확도 지표 RMSE - 낮을수록 정확
def RMSE(y_true, y_pred) :
  return np.sqrt(np.mean((np.array(y_true) - np.array(y_pred)) ** 2))

## score
def score(model, neighbor_size = 0) :
  # 테스트 데이터의 user_id와 movie_id의 쌍을 맞춰 튜플형 원소 리스트 데이터 생성
  id_pairs = zip(x_test["user_id"], x_test["movie_id"])

  # 모든 사용자-영화 쌍에 대해서 주어진 예측 모델에 의해 예측값 계산 및 리스트형 데이터 생성
  y_pred = np.array([model(user, movie, neighbor_size) for (user, movie) in id_pairs])

  # 실제 평점값
  y_true = np.array(x_test["rating"])

  return RMSE(y_true, y_pred)

In [ ]:
# 데이터 셋 생성

x = ratings.copy()
y = ratings["user_id"]

x_train, x_test, y_train, y_test = train_test_split(x, y, test_size = 0.25, stratify = y)

ratings_matrix = x_train.pivot(index = "user_id", columns = "movie_id", values = "rating")

In [ ]:
#코사인 유사도 계산
matrix_dummy = ratings_matrix.copy().fillna(0)

user_similarity = cosine_similarity(matrix_dummy, matrix_dummy)
user_similarity = pd.DataFrame(user_similarity, index = ratings_matrix.index, columns = ratings_matrix.index)

In [ ]:
# KNN
def CF_knn(user_id, movie_id, neighbor_size = 0) :
  if movie_id in ratings_matrix.columns :
    similarity_scores = user_similarity[user_id].copy()
    movie_ratings = ratings_matrix[movie_id].copy()

    none_rating_idx = movie_ratings[movie_ratings.isnull()].index
    movie_ratings = movie_ratings.dropna()
    similarity_scores = similarity_scores.drop(none_rating_idx)

    if neighbor_size == 0 :
      mean_rating = np.dot(similarity_scores, movie_ratings) / similarity_scores.sum()

    else :
      # 평가하지 않은 사용자들 제거했을 때 자기 자신만 남아 있는 경우
      if len(similarity_scores) > 1 :
        # 이웃 개수를 10개로 설정했는데 유사도 있는 사람들(비교할 수 있는 사람들)이 5명이면?
        # 비교할 개수를 줄여주어야 함
        neighbor_size = min(neighbor_size, len(similarity_scores))

        similarity_scores = np.array(similarity_scores)
        movie_ratings = np.array(movie_ratings)

        # 오름차순으로 인덱스값 뽑아내기
        user_idx = np.argsort(similarity_scores)

        similarity_scores = similarity_scores[user_idx][-neighbor_size:]
        movie_ratings = movie_ratings[user_idx][-neighbor_size:]

        mean_rating = np.dot(similarity_scores, movie_ratings) / similarity_scores.sum()

      # 유사도가 1개인 경우
      else :
        mean_rating = 3.0

  else :
      mean_rating = 3.0

  return mean_rating

In [ ]:
# KNN 정확도 계산
score(CF_knn, neighbor_size = 30)
# 1.0109365058919921 정도

1.0136880942516162

In [ ]:
# 실제 주어진 사용자에 대해 추천 받는 기능 구현

## 이전에는 train 데이터를 이용해 pivot 테이블을 생성했는데
## 이번에는 full-matrix를 이용해 전체 데이터로 full-matrix 생성
## 실제 추천할 때는 train 데이터와 test 데이터를 나눌 필요가 없기 때문
rating_matrix = ratings.pivot_table(values = "rating", index = "user_id", columns = "movie_id")

## rating_matrix를 이용해 사용자 간 유사도 구하기
matrix_dummy = rating_matrix.copy().fillna(0)
user_similarity = cosine_similarity(matrix_dummy, matrix_dummy)
user_similarity = pd.DataFrame(user_similarity, index = rating_matrix.index, columns = rating_matrix.index)

In [ ]:
## 실제 추천하는 로직 구현

### n_items : 몇 개 추천받을지
def recom_movie(user_id, n_items, neighbor_size = 30) :
  # 특정 유저의 무비 평가들
  user_movie = rating_matrix.loc[user_id].copy()

  for movie in rating_matrix.columns :
    # null이 아니라면 유저가 이미 영화를 본 것이기 때문에 추천 리스트에서 제외해야 한다
    if pd.notnull(user_movie.loc[movie]) :
      user_movie.loc[movie] = 0

    else :
      user_movie.loc[movie] = CF_knn(user_id, movie, neighbor_size)

  movie_sort = user_movie.sort_values(ascending = False)[:n_items]

  recom_movies = movies.loc[movie_sort.index]
  recommendations = recom_movies["title"]

  return recommendations

In [ ]:
recom_movie(729, 5, 30)

<ipython-input-53-c417a541884c>:30: RuntimeWarning: invalid value encountered in double_scalars
  mean_rating = np.dot(similarity_scores, movie_ratings) / similarity_scores.sum()


movie_id
1612    Leading Man, The (1996)
1293            Star Kid (1997)
1368     Mina Tannenbaum (1994)
22            Braveheart (1995)
1443           8 Seconds (1994)
Name: title, dtype: object

## 3.5. 최적의 이웃 크기 결정
* 집단의 크기가 너무 크면 best-seller 방식과 같이 개인의 취향이 반영되는 정도가 낮다
* 집단의 크기가 너무 작으면 신뢰성이 떨어짐

In [3]:
import os
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.metrics.pairwise import cosine_similarity

# 데이터 불러오기
base_src = "drive/MyDrive/RecoSys/python-recosys/Data"

## user 데이터
u_user_src = os.path.join(base_src, "u.user")
u_cols = ["user_id", "age", "sex", "occupation", "zipcode"]
users = pd.read_csv(u_user_src, sep = "|", names = u_cols, encoding = "latin-1")
users = users.set_index("user_id")

## movie 데이터
u_item_src = os.path.join(base_src, "u.item")
i_cols = ['movie_id','title','release date','video release date',
          'IMDB URL', 'unknown', 'Action', 'Adventure','Animation',
          'ChildrenI\'s','Comedy','Crime','Documentary','Drama','Fantasy',
          'Film-Noir', 'Horror', 'Musical' ,'Mystery','Romance','Sci-Fi','Thriller', 'War','Western']
movies = pd.read_csv(u_item_src, sep = "|", names = i_cols, encoding = "latin-1")
movies = movies.set_index("movie_id")

## ratings 데이터
u_ratings_src = os.path.join(base_src, "u.data")
r_cols = ["user_id", "movie_id", "rating", "timestamp"]
ratings = pd.read_csv(u_ratings_src, sep = "\t", names = r_cols, encoding = "latin-1")


# 필요한 함수 정의

## 정확도 지표 RMSE - 낮을수록 정확
def RMSE(y_true, y_pred) :
  return np.sqrt(np.mean((np.array(y_true) - np.array(y_pred)) ** 2))

## score
def score(model, neighbor_size = 0) :
  # 테스트 데이터의 user_id와 movie_id의 쌍을 맞춰 튜플형 원소 리스트 데이터 생성
  id_pairs = zip(x_test["user_id"], x_test["movie_id"])

  # 모든 사용자-영화 쌍에 대해서 주어진 예측 모델에 의해 예측값 계산 및 리스트형 데이터 생성
  y_pred = np.array([model(user, movie, neighbor_size) for (user, movie) in id_pairs])

  # 실제 평점값
  y_true = np.array(x_test["rating"])

  return RMSE(y_true, y_pred)


# 데이터 셋 생성

x = ratings.copy()
y = ratings["user_id"]

x_train, x_test, y_train, y_test = train_test_split(x, y, test_size = 0.25, stratify = y)

ratings_matrix = x_train.pivot(index = "user_id", columns = "movie_id", values = "rating")


#코사인 유사도 계산
matrix_dummy = ratings_matrix.copy().fillna(0)

user_similarity = cosine_similarity(matrix_dummy, matrix_dummy)
user_similarity = pd.DataFrame(user_similarity, index = ratings_matrix.index, columns = ratings_matrix.index)


# KNN
def CF_knn(user_id, movie_id, neighbor_size = 0) :
  # train/test set 분할에 따라 rating_matrix에 해당 영화가 있는지 확인
  if movie_id in ratings_matrix.columns :
    # 주어진 사용자와 다른 사용자 간의 유사도 추출
    similarity_scores = user_similarity[user_id].copy()
    # 주어진 영화와 다른 사용자 간의 유사도 추출
    movie_ratings = ratings_matrix[movie_id].copy()

    # 주어진 영화에 대해 평가하지 않은 사용자들을 가중평균계산에서 제외하기 위해 추출
    none_rating_idx = movie_ratings[movie_ratings.isnull()].index
    # 주어진 영화에 대해 평가하지 않은 사람들 제외
    movie_ratings = movie_ratings.dropna()
    # 주어진 영화를 평가하지 않은 사용자와의 유사도 제거
    similarity_scores = similarity_scores.drop(none_rating_idx)

    if neighbor_size == 0 :
      mean_rating = np.dot(similarity_scores, movie_ratings) / similarity_scores.sum()

    else :
      # 평가하지 않은 사용자들 제거했을 때 자기 자신만 남아 있는 경우
      if len(similarity_scores) > 1 :
        # 이웃 개수를 10개로 설정했는데 유사도 있는 사람들(비교할 수 있는 사람들)이 5명이면?
        # 비교할 개수를 줄여주어야 함
        neighbor_size = min(neighbor_size, len(similarity_scores))

        similarity_scores = np.array(similarity_scores)
        movie_ratings = np.array(movie_ratings)

        # 오름차순으로 인덱스값 뽑아내기
        user_idx = np.argsort(similarity_scores)

        similarity_scores = similarity_scores[user_idx][-neighbor_size:]
        movie_ratings = movie_ratings[user_idx][-neighbor_size:]

        mean_rating = np.dot(similarity_scores, movie_ratings) / similarity_scores.sum()

      # 유사도가 1개인 경우
      else :
        mean_rating = 3.0

  else :
      mean_rating = 3.0

  return mean_rating

In [4]:
# neighbor_size가 10, 20, 30, 40, 50, 60 인 경우에 대해 정확도 확인
for neighbor_size in [10, 20, 30, 40, 50, 60] :
  print("Neighbor Size = %d : RMSE = %.4f" %(neighbor_size, score(CF_knn, neighbor_size)))
## 30과 40 사이가 적당하다는 사실을 알 수 있다

Neighbor Size = 10 : RMSE = 1.0139


KeyboardInterrupt: ignored

## 3.6. 사용자의 평가경향을 고려한 CF
* 같은 평점 다른 의미  
  * 평가를 높게 하는 사람, 평가를 낮게 하는 사람
* 집단과 사용자 간의 차이를 조정하는 고정치 필요

> 1. 각 사용자 평점 평균 계산
> 2. (각 상품의) 평점 -> 각 사용자의 평균에서의 차이로 변환  
> → 평점 - 해당 사용자의 평점 평균
> 3. 평점 편차의 예측값 계산  
> → 평가값 = 평점 편차 × 다른 사용자 유사도
> 4. 실제 예측값 = 평점편차 예측값 + 평점 평균


In [5]:
# 실제 주어진 사용자에 대해 추천 받는 기능 구현

## 이전에는 train 데이터를 이용해 pivot 테이블을 생성했는데
## 이번에는 full-matrix를 이용해 전체 데이터로 full-matrix 생성
## 실제 추천할 때는 train 데이터와 test 데이터를 나눌 필요가 없기 때문
rating_matrix = ratings.pivot_table(values = "rating", index = "user_id", columns = "movie_id")

## rating_matrix를 이용해 사용자 간 유사도 구하기
matrix_dummy = rating_matrix.copy().fillna(0)
user_similarity = cosine_similarity(matrix_dummy, matrix_dummy)
user_similarity = pd.DataFrame(user_similarity, index = rating_matrix.index, columns = rating_matrix.index)

In [9]:
# full-matrix에서 각 사용자의 평점의 평균 구하기
## 사용자 별로 리뷰한 평점들의 평균
rating_mean = rating_matrix.mean(axis = 1)

#영화 평점과 각 사용자의 평균과의 차이(평점편차) 구하기
## 평점에서 각 사용자별 평균 점수 빼주기
rating_bias = (rating_matrix.T -rating_mean).T

In [30]:
rating_bias

movie_id,1,2,3,4,5,6,7,8,9,10,...,1673,1674,1675,1676,1677,1678,1679,1680,1681,1682
user_id,,,,,,,,,,,,,,,,,,,,,
1,1.389706,-0.610294,0.389706,-0.610294,-0.610294,1.389706,0.389706,-2.610294,1.389706,-0.610294,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,0.290323,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,-1.709677,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5,1.125714,0.125714,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
939,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.734694,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
940,NaN,NaN,NaN,-1.457944,NaN,NaN,0.542056,1.542056,-0.457944,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
941,0.954545,NaN,NaN,NaN,NaN,NaN,-0.045455,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [73]:
movie_ratings = rating_bias[1][:10].copy()
none_rating_idx = movie_ratings[movie_ratings.isnull()].index
movie_ratings = movie_ratings.drop(none_rating_idx)
movie_ratings = np.array(movie_ratings)
print(movie_ratings)

similarity_scores = user_similarity[1][:10].copy()
similarity_scores = similarity_scores.drop(none_rating_idx)
print(similarity_scores)
similarity_scores = np.array(similarity_scores)
print(similarity_scores)
user_idx = np.argsort(similarity_scores)
print(user_idx)

print(movie_ratings[user_idx])
print(movie_ratings[user_idx][-2:])
# movie_ratings = movie_ratings[np.array([1, 2, 3, 4])][-2:]
# movie_ratings

[ 1.38970588  0.29032258  1.12571429  0.36492891 -0.20652174]
user_id
1     1.000000
2     0.166931
5     0.378475
6     0.430239
10    0.376544
Name: 1, dtype: float64
[1.         0.16693098 0.37847518 0.43023944 0.37654381]
[1 4 2 3 0]
[ 0.29032258 -0.20652174  1.12571429  0.36492891  1.38970588]
[0.36492891 1.38970588]


In [76]:
# 사용자 평가 경향을 고려한 함수
def CF_knn_bias(user_id, movie_id, neighbor_size = 0) :
  if movie_id in rating_bias.columns :
    similarity_scores = user_similarity[user_id].copy()
    movie_ratings = rating_bias[movie_id].copy()

    none_rating_idx = movie_ratings[movie_ratings.isnull()].index
    movie_ratings = movie_ratings.drop(none_rating_idx)
    similarity_scores = similarity_scores.drop(none_rating_idx)

    if neighbor_size == 0 :
      prediction = np.dot(similarity_scores, movie_ratings) / similarity_scores.sum()
      # 사용자의 평점 평균 다시 더해주기
      prediction = prediction + rating_mean[user_id]

    else :
      if len(similarity_scores) > 1 :
        neighbor_size = min(neighbor_size, len(similarity_scores))

        similarity_scores = np.array(similarity_scores)
        movie_ratings = np.array(movie_ratings)

        # 배열의 원소를 정렬하지 않고, 정렬된 순서로 원소의 인덱스를 얻을 수 있다
        # 유사도가 낮은 순서에서 높은 순서로 정렬된 인덱스를 얻을 수 있다
        user_idx = np.argsort(similarity_scores)

        similarity_scores = similarity_scores[user_idx][-neighbor_size:]
        movie_ratings = movie_ratings[user_idx][-neighbor_size:]

        prediction = np.dot(similarity_scores, movie_ratings) / similarity_scores.sum()
        prediction = prediction + rating_mean[user_id]

      else :
        # 이전에는 그냥 3.0을 넘겨주었지만
        # 이번에는 사용자의 평가 경향을 고려하는 것이기 때문에
        # 사용자가 남긴 평점들의 평균으로 설정
        prediction = rating_mean[user_id]

  else :
    prediction = rating_mean[user_id]

  return prediction

In [77]:
score(CF_knn_bias, 30)
# 0.8402064766090754
# 사용자의 평가 경향을 고려했을 때 성능이 개선되었음

0.8402064766090754

## 3.7. 이 외의 CF 정확도 개선 방법
### 3.7.1. 신뢰도 가중 방법
* 어떤 사용자A와 0.8로 같은 유사도를 가진 사용자 B와 C가 있다고 가정했을 때,  
  * 사용자B는 사용자A와 공통으로 평가한 아이템 2개
  * 사용자C는 사용자A와 공통으로 평가한 아이템 10개

  → 사용자B 보다 사용자C의 신뢰도가 더 높다
* 따라서 신뢰도에 따라 유사도에 가중 하자
* 예측값은 매우 민감하기 때문에
  * 공통의 아이템 수를 직접적으로 반영을 하기보다
  * 공통으로 평가한 아이템의 수가 일정 개수 이상인 사용자들의 집단만 활용


In [78]:
import os
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.metrics.pairwise import cosine_similarity

# 데이터 불러오기
base_src = "drive/MyDrive/RecoSys/python-recosys/Data"

## user 데이터
u_user_src = os.path.join(base_src, "u.user")
u_cols = ["user_id", "age", "sex", "occupation", "zipcode"]
users = pd.read_csv(u_user_src, sep = "|", names = u_cols, encoding = "latin-1")
users = users.set_index("user_id")

## movie 데이터
u_item_src = os.path.join(base_src, "u.item")
i_cols = ['movie_id','title','release date','video release date',
          'IMDB URL', 'unknown', 'Action', 'Adventure','Animation',
          'ChildrenI\'s','Comedy','Crime','Documentary','Drama','Fantasy',
          'Film-Noir', 'Horror', 'Musical' ,'Mystery','Romance','Sci-Fi','Thriller', 'War','Western']
movies = pd.read_csv(u_item_src, sep = "|", names = i_cols, encoding = "latin-1")
movies = movies.set_index("movie_id")

## ratings 데이터
u_ratings_src = os.path.join(base_src, "u.data")
r_cols = ["user_id", "movie_id", "rating", "timestamp"]
ratings = pd.read_csv(u_ratings_src, sep = "\t", names = r_cols, encoding = "latin-1")


# 필요한 함수 정의

## 정확도 지표 RMSE - 낮을수록 정확
def RMSE(y_true, y_pred) :
  return np.sqrt(np.mean((np.array(y_true) - np.array(y_pred)) ** 2))

## score
def score(model, neighbor_size = 0) :
  # 테스트 데이터의 user_id와 movie_id의 쌍을 맞춰 튜플형 원소 리스트 데이터 생성
  id_pairs = zip(x_test["user_id"], x_test["movie_id"])

  # 모든 사용자-영화 쌍에 대해서 주어진 예측 모델에 의해 예측값 계산 및 리스트형 데이터 생성
  y_pred = np.array([model(user, movie, neighbor_size) for (user, movie) in id_pairs])

  # 실제 평점값
  y_true = np.array(x_test["rating"])

  return RMSE(y_true, y_pred)


# 데이터 셋 생성

x = ratings.copy()
y = ratings["user_id"]

x_train, x_test, y_train, y_test = train_test_split(x, y, test_size = 0.25, stratify = y)

ratings_matrix = x_train.pivot(index = "user_id", columns = "movie_id", values = "rating")


#코사인 유사도 계산
matrix_dummy = ratings_matrix.copy().fillna(0)

user_similarity = cosine_similarity(matrix_dummy, matrix_dummy)
user_similarity = pd.DataFrame(user_similarity, index = ratings_matrix.index, columns = ratings_matrix.index)


# KNN
def CF_knn(user_id, movie_id, neighbor_size = 0) :
  # train/test set 분할에 따라 rating_matrix에 해당 영화가 있는지 확인
  if movie_id in ratings_matrix.columns :
    # 주어진 사용자와 다른 사용자 간의 유사도 추출
    similarity_scores = user_similarity[user_id].copy()
    # 주어진 영화와 다른 사용자 간의 유사도 추출
    movie_ratings = ratings_matrix[movie_id].copy()

    # 주어진 영화에 대해 평가하지 않은 사용자들을 가중평균계산에서 제외하기 위해 추출
    none_rating_idx = movie_ratings[movie_ratings.isnull()].index
    # 주어진 영화에 대해 평가하지 않은 사람들 제외
    movie_ratings = movie_ratings.dropna()
    # 주어진 영화를 평가하지 않은 사용자와의 유사도 제거
    similarity_scores = similarity_scores.drop(none_rating_idx)

    if neighbor_size == 0 :
      mean_rating = np.dot(similarity_scores, movie_ratings) / similarity_scores.sum()

    else :
      # 평가하지 않은 사용자들 제거했을 때 자기 자신만 남아 있는 경우
      if len(similarity_scores) > 1 :
        # 이웃 개수를 10개로 설정했는데 유사도 있는 사람들(비교할 수 있는 사람들)이 5명이면?
        # 비교할 개수를 줄여주어야 함
        neighbor_size = min(neighbor_size, len(similarity_scores))

        similarity_scores = np.array(similarity_scores)
        movie_ratings = np.array(movie_ratings)

        # 오름차순으로 인덱스값 뽑아내기
        user_idx = np.argsort(similarity_scores)

        similarity_scores = similarity_scores[user_idx][-neighbor_size:]
        movie_ratings = movie_ratings[user_idx][-neighbor_size:]

        mean_rating = np.dot(similarity_scores, movie_ratings) / similarity_scores.sum()

      # 유사도가 1개인 경우
      else :
        mean_rating = 3.0

  else :
      mean_rating = 3.0

  return mean_rating


# 실제 주어진 사용자에 대해 추천 받는 기능 구현

## 이전에는 train 데이터를 이용해 pivot 테이블을 생성했는데
## 이번에는 full-matrix를 이용해 전체 데이터로 full-matrix 생성
## 실제 추천할 때는 train 데이터와 test 데이터를 나눌 필요가 없기 때문
rating_matrix = ratings.pivot_table(values = "rating", index = "user_id", columns = "movie_id")

## rating_matrix를 이용해 사용자 간 유사도 구하기
matrix_dummy = rating_matrix.copy().fillna(0)
user_similarity = cosine_similarity(matrix_dummy, matrix_dummy)
user_similarity = pd.DataFrame(user_similarity, index = rating_matrix.index, columns = rating_matrix.index)


# full-matrix에서 각 사용자의 평점의 평균 구하기
## 사용자 별로 리뷰한 평점들의 평균
rating_mean = rating_matrix.mean(axis = 1)


#영화 평점과 각 사용자의 평균과의 차이(평점편차) 구하기
## 평점에서 각 사용자별 평균 점수 빼주기
rating_bias = (rating_matrix.T -rating_mean).T

In [91]:
# 신뢰도 반영하기

## rating_matrix > 0 : boolean값 반환
## np.array로 만들고 float를 적용하면 0과 1로 반환
rating_binary_1 = np.array(rating_matrix>0).astype(float)
rating_binary_2 = rating_binary_1.T

## counts[i][i]는 사용자 i가 평가한 영화의 개수
## counts[i][j]는 사용자 i와 사용자 j가 공통으로 평가한 영화의 개수
counts = np.dot(rating_binary_1, rating_binary_2)
counts = pd.DataFrame(counts, index = rating_matrix.index, columns = rating_matrix.index).fillna(0)

In [92]:
counts

user_id,1,2,3,4,5,6,7,8,9,10,...,934,935,936,937,938,939,940,941,942,943
user_id,,,,,,,,,,,,,,,,,,,,,
1,272.0,18.0,8.0,7.0,80.0,96.0,145.0,34.0,5.0,77.0,...,77.0,15.0,48.0,19.0,36.0,15.0,52.0,10.0,27.0,83.0
2,18.0,62.0,9.0,7.0,5.0,32.0,18.0,6.0,6.0,16.0,...,15.0,14.0,33.0,20.0,24.0,11.0,17.0,7.0,12.0,9.0
3,8.0,9.0,54.0,13.0,1.0,10.0,14.0,7.0,2.0,8.0,...,3.0,2.0,16.0,6.0,10.0,2.0,15.0,4.0,9.0,2.0
4,7.0,7.0,13.0,24.0,2.0,6.0,12.0,8.0,2.0,4.0,...,3.0,1.0,8.0,6.0,7.0,1.0,10.0,4.0,8.0,4.0
5,80.0,5.0,1.0,2.0,175.0,42.0,102.0,23.0,4.0,35.0,...,58.0,6.0,15.0,5.0,21.0,7.0,28.0,6.0,17.0,59.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
939,15.0,11.0,2.0,1.0,7.0,14.0,16.0,4.0,1.0,6.0,...,7.0,18.0,23.0,10.0,29.0,49.0,5.0,6.0,3.0,12.0
940,52.0,17.0,15.0,10.0,28.0,54.0,68.0,18.0,6.0,51.0,...,45.0,7.0,26.0,13.0,18.0,5.0,107.0,7.0,23.0,28.0
941,10.0,7.0,4.0,4.0,6.0,11.0,7.0,7.0,3.0,6.0,...,3.0,6.0,16.0,7.0,14.0,6.0,7.0,22.0,4.0,5.0


In [101]:
# from operator import ne
def CF_knn_bias_sig(user_id, movie_id, neighbor_size = 0) :
  if movie_id in rating_bias :
    similarity_scores = user_similarity[user_id].copy()
    movie_ratings = rating_bias[movie_id].copy()

    # 영화 평점, 유사도 전처리
    ## 영화에 대해 평점이 있으면 False, 없으면 True
    no_rating = movie_ratings.isnull()
    ## 특정 사용자와 공통으로 평가한 영화들의 개수
    common_counts = counts[user_id]
    ## 공통으로 평가한 영화의 개수가
    # 미리 정해진 숫자 SIG_LEVEL 보다 작으면 True
    low_significance = common_counts < SIG_LEVEL
    ## 영화를 평가하지 않았거나, 공통으로 평가한 영화의 개수가 너무 작으면 제외
    none_rating_idx = movie_ratings[no_rating | low_significance].index

    movie_ratings = movie_ratings.drop(none_rating_idx)
    similarity_scores = similarity_scores.drop(none_rating_idx)

    if neighbor_size == 0 :
      prediction = np.dot(similarity_scores, movie_ratings) / similarity_scores.sum()
      prediction = prediction + rating_mean[user_id]

    else :
      # 이전까지는 1보다 큰 것으로 판별했는데
      # 현재 영화를 평가한 사용자 수가 미리 정한 최소 사용자 수보다 큰 것으로 판별
      if len(similarity_scores) > MIN_RATINGS :
        neighbor_size = min(neighbor_size, len(similarity_scores))

        similarity_scores = np.array(similarity_scores)
        movie_ratings = np.array(movie_ratings)

        user_idx = np.argsort(similarity_scores)
        similarity_scores = similarity_scores[user_idx][-neighbor_size:]
        movie_ratings = movie_ratings[user_idx][-neighbor_size:]

        prediction = np.dot(similarity_scores, movie_ratings) / similarity_scores.sum()
        prediction = prediction + rating_mean[user_id]

      else :
        prediction = rating_mean[user_id]

  else :
    prediction = rating_mean[user_id]

  return prediction

In [103]:
# 공통으로 평가한 영화 개수
SIG_LEVEL = 3
# 영화 리뷰 최소 몇 개 이상
MIN_RATINGS = 3
score(CF_knn_bias_sig, 30)
# 0.8387156925670546
# 평가경향만 고려했을 때보다 조금 더 개선됨

0.8387156925670546

In [100]:
movie_ratings = rating_bias[1].copy()
no_rating = movie_ratings.isnull()
print(no_rating)
common_counts = counts[1]
low_significance = common_counts < 11
print(low_significance)
# none_rating_idx = movie_ratings[no_rating | low_significance].index
print(movie_ratings[no_rating | low_significance])

user_id
1      False
2      False
3       True
4       True
5      False
       ...  
939     True
940     True
941    False
942     True
943     True
Name: 1, Length: 943, dtype: bool
user_id
1      False
2      False
3       True
4       True
5      False
       ...  
939    False
940    False
941     True
942    False
943    False
Name: 1, Length: 943, dtype: bool
user_id
3           NaN
4           NaN
7           NaN
8           NaN
9           NaN
         ...   
939         NaN
940         NaN
941    0.954545
942         NaN
943         NaN
Name: 1, Length: 504, dtype: float64
